#Q1: Implement Multinomial Naïve Bayes classifier (in-built) for detection of messages as spam or ham?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
path='/content/drive/MyDrive/Colab Notebooks/ML/assignment 7/message.txt'
with open(path) as f:
    data=f.readlines()
len(data)

5574

In [ ]:
import nltk
import re
for i in range(len(data)):
    data[i]=re.sub('[^a-zA-Z0-9]',' ',data[i])
    data[i]=data[i].lower()

In [ ]:
data[0]

'ham go until jurong point  crazy   available only in bugis n great world la e buffet    cine there got amore wat    '

In [ ]:
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
token_word=[]
for i in range(len(data)):
    token_word.append(nltk.word_tokenize(data[i]))
len(token_word)
#token_word

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


5574

In [ ]:
x=[]
y=[]
for i in range(len(token_word)):
    x.append(token_word[i][1:])
    y.append(token_word[i][0])
print(len(x),len(y))

5574 5574


In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
words=[]
#for i in range(len(x)):
ps=PorterStemmer()
let=WordNetLemmatizer()
stp=stopwords.words(fileids='english')
for i in range(len(x)):
    sent=[let.lemmatize(w) for w in x[i] if w not in stp]
    words.append(' '.join(sent))
len(words)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


5574

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
x_bow=cv.fit_transform(words).toarray()
x_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
x_bow.shape

(5574, 8046)

In [ ]:
y_=np.array(y)
y_.shape

(5574,)

In [ ]:
np.unique(y_,return_counts=True)

(array(['ham', 'spam'], dtype='<U4'), array([4827,  747]))

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_bow,y_,test_size=0.15)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(4737, 8046) (4737,)
(837, 8046) (837,)


In [ ]:
y_tr=np.zeros((y_train.shape[0]))
print(y_tr.shape)
for i in range(y_train.shape[0]): 
    if y_train[i]=='spam':
        y_tr[i]=1
print(y_tr.shape)

from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()
nb.fit(x_train,y_train)
nb.score(x_test,y_test)

(4737,)
(4737,)


0.977299880525687

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(n_components=50)
x_bow=pca.fit_transform(x_bow)
print(x_bow.shape)
pca.explained_variance_ratio_

(5574, 50)


array([0.02915715, 0.01675639, 0.01425048, 0.01034979, 0.00965049,
       0.00784804, 0.00716174, 0.00703686, 0.00634851, 0.00608856,
       0.00589396, 0.0057784 , 0.00564784, 0.00541787, 0.00536031,
       0.00525536, 0.00499184, 0.00490555, 0.004824  , 0.0047624 ,
       0.00465855, 0.00457682, 0.00441044, 0.00431276, 0.00425185,
       0.00407315, 0.004043  , 0.00401545, 0.0039469 , 0.00387204,
       0.00381875, 0.00375755, 0.00369151, 0.00363681, 0.00360063,
       0.00356522, 0.00349755, 0.00347587, 0.00337455, 0.00332744,
       0.00329668, 0.00327568, 0.00324748, 0.00318567, 0.00314819,
       0.00310555, 0.00308178, 0.00305976, 0.00301648, 0.00297654])

In [ ]:
def prior_prob(y_train,label):  #P(A)
    total=y_train.shape[0]
    count=np.sum(y_train==label)

    return count/total

    
def cond_prob(x_train,y_train,feature_col,feature_val,label):
    x_filter=x_train[y_train==label]

    den=x_filter.shape[0]
    num=np.sum(x_filter[:,feature_col]==feature_val)

    col=x_filter.shape[1]

    #print(num,den,col)
    return (1+num)/(den+col)


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_bow,y_,test_size=0.15)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(4737, 50) (4737,)
(837, 50) (837,)


In [ ]:
lab=list(np.unique(y_train))
lab

['ham', 'spam']

In [ ]:
def predict(x_test,x_train,y_train):
    output=[]
    n_features=x_train.shape[1]
    #print(n_features)
    lab=list(np.unique(y_train))
    #for row in x_test:
    out=[]#np.zeros((len(lab)))
    for l in range(len(lab)):
        prob=prior_prob(y_train,lab[l])
        #print(prob)
        for fet in range(n_features):
            #print(fet)
            cond=cond_prob(x_train,y_train,fet,x_test[fet],lab[l])
            prob*=cond
        out.append(prob)
        #print('out',out)
        #output.append(out)
    return out

In [ ]:
y_pred=[]
n=x_test.shape[0]
print(n)
for i in range(n):
    pre=predict(x_test[i],x_train,y_train)
    y_pred.append(np.array(pre))
len(y_pred)

837


837

In [ ]:
y_final=np.argmax(y_pred,axis=1)

In [ ]:
len(y_final)

837

In [ ]:
y_=np.zeros(y_test.shape)
#y_
for i in range(y_test.shape[0]):
    if y_test[i]=='spam':
        y_[i]=1
y_.shape

(837,)

In [ ]:
acc=0
for i in range(y_.shape[0]):
    if y_final[i]==y_[i]:
        acc+=1
print(acc/y_.shape[0])

0.15412186379928317


# Q2: (Gaussian Naïve Bayes Classifier) Implement Gaussian Naïve Bayes Classifier on the Iris dataset from sklearn.datasets using


#Q2: (i) Step-by-step implementation


In [ ]:
from sklearn.datasets import load_iris
data=load_iris()
x=data['data']
y=data['target']
print(x.shape,y.shape)

(150, 4) (150,)


In [ ]:
labels=list(np.unique(y))
print(labels)
output=data['target_names']
output

[0, 1, 2]


array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [ ]:
import random
x_comb=np.hstack((x,y.reshape(-1,1)))
np.random.shuffle(x_comb)
x_comb

array([[5.4, 3.9, 1.7, 0.4, 0. ],
       [6.4, 2.8, 5.6, 2.1, 2. ],
       [5.1, 3.8, 1.6, 0.2, 0. ],
       [6.5, 3. , 5.5, 1.8, 2. ],
       [6. , 2.2, 5. , 1.5, 2. ],
       [4.9, 3.6, 1.4, 0.1, 0. ],
       [5.8, 4. , 1.2, 0.2, 0. ],
       [5.4, 3.9, 1.3, 0.4, 0. ],
       [5.5, 2.4, 3.8, 1.1, 1. ],
       [5.1, 3.7, 1.5, 0.4, 0. ],
       [6.9, 3.2, 5.7, 2.3, 2. ],
       [6.7, 3.1, 4.7, 1.5, 1. ],
       [6.3, 2.5, 5. , 1.9, 2. ],
       [6.7, 3. , 5.2, 2.3, 2. ],
       [5. , 3.5, 1.3, 0.3, 0. ],
       [6.5, 2.8, 4.6, 1.5, 1. ],
       [4.9, 2.5, 4.5, 1.7, 2. ],
       [5.7, 4.4, 1.5, 0.4, 0. ],
       [6.4, 2.9, 4.3, 1.3, 1. ],
       [4.6, 3.1, 1.5, 0.2, 0. ],
       [5.6, 2.5, 3.9, 1.1, 1. ],
       [7.4, 2.8, 6.1, 1.9, 2. ],
       [7.2, 3. , 5.8, 1.6, 2. ],
       [5.5, 2.6, 4.4, 1.2, 1. ],
       [5.9, 3. , 4.2, 1.5, 1. ],
       [5. , 3.3, 1.4, 0.2, 0. ],
       [6.5, 3. , 5.8, 2.2, 2. ],
       [6.1, 2.9, 4.7, 1.4, 1. ],
       [5.7, 3.8, 1.7, 0.3, 0. ],
       [5.8, 2

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_comb,y,test_size=0.15)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(127, 5) (127,)
(23, 5) (23,)


In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_sc=sc.fit_transform(x_train)
x_test_sc=sc.transform(x_test)
print(x_sc.shape,x_test_sc.shape)
#x_test_sc

(127, 5) (23, 5)


In [ ]:
# row= 0,1,2
# col=colums of train data
def gaussian_train(x,y):
    labels=list(np.unique(y))
    n_features=x.shape[1]
    mean=np.zeros((len(labels),n_features),dtype=np.float64)
    var=np.zeros((len(labels),n_features),dtype=np.float64)
    prior=np.zeros((len(labels)),dtype=np.float64)

    for i in range(len(labels)):
        l=labels[i]
        x_filter=x[y==l]
        print(l)
        #mean[l]=
        mean[l,:]=np.mean(x_filter,axis=0)
        var[l,:]=np.var(x_filter,axis=0)
        prior[l]=(x_filter.shape[0]/x.shape[0])
        #print(np.mean(x_filter,axis=0))
    return mean,var,prior
#print(mean)
#print(var)
#print(prior)

def cond_prob(x,mean,var,feat,lab):
    m=mean[lab][feat]
    v=var[lab][feat]
    num=np.exp(-1/2*((x[feat]-m)**2)/v)
    den=np.sqrt(2*np.pi*v)

    return num/den

#k=12
def predict(x_test,mean,var,prior,labels):
    output=[]
    for row in range(x_test.shape[0]):
        out=np.zeros((len(labels)))
        for i in range(len(labels)):
            prob=prior[i]
            for j in range(x.shape[1]):
                prob*=cond_prob(x_test[row],mean,var,j,i)
                #print('p',prob)
            out[i]=prob
            #print(prob)
        output.append(out)
    return np.array(output)

In [ ]:
mean,var,prior=gaussian_train(x_sc,y_train)
print(mean)
print(var)
print(prior)

0
1
2
[[-0.0617486   0.15067725 -0.12144461 -0.13541125 -0.09621226]
 [ 0.0764377  -0.10848929  0.12749459  0.16296828  0.13542673]
 [-0.01434749 -0.04120685 -0.00590928 -0.02691617 -0.03830251]]
[[0.79335646 1.24677714 0.96214053 0.89753909 1.05301124]
 [1.14264414 0.82595349 0.97315855 1.02424503 0.95575861]
 [1.05287413 0.89359092 1.03287868 1.03182163 0.96301161]]
[0.33070866 0.33070866 0.33858268]


In [ ]:
y_pred=predict(x_test_sc,mean,var,prior,labels)
pred=np.argmax(y_pred,axis=1)
print('Accuracy',end=' ')
np.sum(pred==y_test)/y_test.shape[0]

Accuracy 

0.391304347826087

In [ ]:
y_pred=predict(x_sc,mean,var,prior,labels)
pred=np.argmax(y_pred,axis=1)
print('Accuracy',end=' ')
np.sum(pred==y_train)/y_train.shape[0]

Accuracy 

0.3700787401574803

#Q2: (ii) In-built function

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb=GaussianNB()
gnb.fit(x_sc,y_train)
print("Accuracy",gnb.score(x_test_sc,y_test))

Accuracy 0.4782608695652174


#Q3: (GridSearchCV for KNN) Explore about GridSearchCV toot in scikit-learn. This is a tool that is often used for tuning hyperparameters of machine learning models. Use this tool to find the best value of K for K-NN Classifier using any dataset

In [ ]:
from sklearn.datasets import load_iris
data=load_iris()
x=data['data']
y=data['target']
print(x.shape,y.shape)

(150, 4) (150,)


In [ ]:
labels=list(np.unique(y))
print(labels)
output=data['target_names']
output

[0, 1, 2]


array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [ ]:
import random
x_comb=np.hstack((x,y.reshape(-1,1)))
np.random.shuffle(x_comb)
x_comb

array([[4.7, 3.2, 1.3, 0.2, 0. ],
       [5.8, 2.6, 4. , 1.2, 1. ],
       [6.7, 3.1, 4.7, 1.5, 1. ],
       [5.6, 2.8, 4.9, 2. , 2. ],
       [5. , 3.2, 1.2, 0.2, 0. ],
       [4.8, 3.4, 1.9, 0.2, 0. ],
       [6.3, 3.4, 5.6, 2.4, 2. ],
       [5.5, 2.4, 3.7, 1. , 1. ],
       [6.7, 3.3, 5.7, 2.5, 2. ],
       [6. , 3.4, 4.5, 1.6, 1. ],
       [6.3, 2.5, 4.9, 1.5, 1. ],
       [6. , 3. , 4.8, 1.8, 2. ],
       [6.9, 3.1, 4.9, 1.5, 1. ],
       [7.7, 3. , 6.1, 2.3, 2. ],
       [6.3, 3.3, 4.7, 1.6, 1. ],
       [6.1, 3. , 4.6, 1.4, 1. ],
       [6.6, 2.9, 4.6, 1.3, 1. ],
       [6.9, 3.2, 5.7, 2.3, 2. ],
       [5.7, 3. , 4.2, 1.2, 1. ],
       [5.8, 2.7, 5.1, 1.9, 2. ],
       [6.5, 2.8, 4.6, 1.5, 1. ],
       [6.3, 2.9, 5.6, 1.8, 2. ],
       [4.7, 3.2, 1.6, 0.2, 0. ],
       [5.1, 3.8, 1.9, 0.4, 0. ],
       [5.4, 3. , 4.5, 1.5, 1. ],
       [5.4, 3.9, 1.7, 0.4, 0. ],
       [4.9, 2.4, 3.3, 1. , 1. ],
       [4.9, 3.6, 1.4, 0.1, 0. ],
       [5.6, 2.7, 4.2, 1.3, 1. ],
       [6.3, 3

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_comb,y,test_size=0.15)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(127, 5) (127,)
(23, 5) (23,)


In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_sc=sc.fit_transform(x_train)
x_test_sc=sc.transform(x_test)
print(x_sc.shape,x_test_sc.shape)
#x_test_sc

(127, 5) (23, 5)


In [ ]:
def distance(x1,p):
    return np.sqrt(sum((x1-p)**2))

def knn(x_train,y_train,point,k=5):
    val=[]
    m=x_train.shape[0]

    for i in range(m):
        d=distance(x_train[i],point)
        val.append([d,y_train[i]])
    #print(val)
    #print('')
    val=sorted(val)
    #print(val[:k])

    return np.array(val[:k])
v=knn(x_train,y_train,x_test[0])
v

array([[0.38729833, 0.        ],
       [0.43588989, 1.        ],
       [0.46904158, 1.        ],
       [0.64807407, 1.        ],
       [0.65574385, 0.        ]])

In [ ]:
va,c=np.unique(v[:,1],return_counts=True)
print(va,c)
va[np.argmax(c)]
y_pred=[]
for row in range(x_test_sc.shape[0]):
    dis=knn(x_sc,y_train,x_test_sc[row],k=5)
    va,c=np.unique(dis[:,1],return_counts=True)
    y_pred.append(va[np.argmax(c)])
print(y_pred)
print('accuracy',end=' ')
np.sum(y_test==y_pred)/y_test.shape[0]

[0. 1.] [2 3]
[0.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 2.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 2.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0]
accuracy 

0.34782608695652173

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier()
kn.fit(x_sc, y_train)
kn.score(x_test_sc,y_test)
#y_pred = classifier.predict(X_test)
#cm = confusion_matrix(y_test, y_pred)
#print(cm)
#accuracy_score(y_test, y_pred)

0.34782608695652173

In [ ]:
from sklearn.model_selection import GridSearchCV
param={'n_neighbors':[5,6,7,8,9,10,11,12,13,14,15],
       'weights':['uniform','distance'],
       'metric':['euclidean','manhattan']}
grid_search = GridSearchCV(estimator = kn,
                           param_grid = param,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search.fit(x_sc, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print(best_accuracy)
print(best_parameters)

0.3467948717948718
{'metric': 'euclidean', 'n_neighbors': 12, 'weights': 'uniform'}
